In [1]:
import tensorflow as tf
import numpy as np
import random

In [2]:
f = open('C:\\Users\\dudemonkeys\\Documents\\Python Scripts\\Studies\\drug_names.txt','r')
input_text = f.read()
chars = list(set(input_text))
word_list = input_text.split('\n')
word_list = list(filter(lambda x : len(x) >= 5, word_list))

['maxidex',
 'enemeez',
 'peptamen prebio',
 'alfamino junior',
 'erbitux',
 'vesicare',
 'acuvail',
 'incruse ellipta',
 'xtandi',
 'synthroid tablets',
 'vyvanse',
 'analpram hc cream ',
 'gammagard',
 'donnatal tablet',
 'corifact',
 'linzess capsules',
 'afinitor',
 'procardia',
 'lenvima',
 'lioresal injectable',
 'maxalt-mlt',
 'sporanox casules',
 'inspra',
 'primaxin im',
 'omnitrope ',
 'tafinlar',
 'travatan z ',
 'tranxene',
 'prenate star',
 'cleocin phosphate',
 'technivie',
 'auvi-q',
 'zyprexa',
 'flector patch',
 'soliqua',
 'anusol-hc suppository',
 'hycamtin injection',
 'zetonna nasal aerosol',
 'promacta',
 'invokamet xr',
 'diflucan',
 'toviaz',
 'zyprexa relprevv',
 'alfamino infant',
 'epivir tablets',
 'h.p. acthar gel',
 'xanax',
 'ancobon caps',
 'eylea injection',
 'cardizem la',
 'oseni',
 'ridaura',
 'canasa',
 'norvasc',
 'aromasin',
 'sculptra',
 'perjeta',
 'prempro',
 'picato gel',
 'innopran xl',
 'tegretol',
 'aerochamber plus flow-vu mouthpiece/mask'

In [3]:
chars.index('a')

31

In [4]:
#hyperameters
hidden_size = 100
seq_length = min([len(word) for word in word_list])
learning_rate = 1e-1

vocab_size = len(chars)
batch_size = 1

In [22]:
num_epochs = 1

In [6]:
Wxh = tf.Variable(tf.random_normal([hidden_size, vocab_size],  stddev=0.1))
Whh = tf.Variable(tf.random_normal([hidden_size, hidden_size], stddev=0.1))
Why = tf.Variable(tf.random_normal([vocab_size, hidden_size],  stddev= 0.1))

bh = tf.Variable(tf.zeros([hidden_size, 1]))
by = tf.Variable(tf.zeros([vocab_size, 1]))
tf.trainable_variables()

[<tf.Variable 'Variable:0' shape=(100, 38) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'Variable_2:0' shape=(38, 100) dtype=float32_ref>,
 <tf.Variable 'Variable_3:0' shape=(100, 1) dtype=float32_ref>,
 <tf.Variable 'Variable_4:0' shape=(38, 1) dtype=float32_ref>]

In [7]:
X = tf.placeholder(tf.float32, [vocab_size, seq_length])
y_ = tf.placeholder(tf.float32, [vocab_size, seq_length])
init_state = tf.placeholder(tf.float32, [hidden_size, batch_size])

input_series = tf.unstack(X,axis=1)
labels_series = tf.unstack(y_, axis=1)
labels_series = [tf.reshape(label, [vocab_size, batch_size]) for label in labels_series]

In [ ]:
current_state = init_state
states_series = []


for current_input in input_series:
    current_input = tf.reshape(current_input, [vocab_size, batch_size])
    next_state = tf.tanh(tf.matmul(Wxh, current_input)+ tf.matmul(Whh, current_state)+bh)
    states_series.append(next_state)
    current_state = next_state

logits_series = [tf.matmul(Why, state)+by for state in states_series]
predictions_series = [tf.nn.softmax(logit) for logit in logits_series]

losses = [tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logits_series, labels_series)]
total_loss = tf.reduce_mean(losses)
#tf.trainable_variables()
train_step = tf.train.AdagradOptimizer(1e-4).minimize(total_loss)

In [9]:
def one_hot(character):
    vec = np.zeros((vocab_size,1),dtype=np.float32)
    vec[chars.index(character)] = 1
    return vec

In [10]:
def word_mat(word):
    mat = np.array([one_hot(char) for char in word]).transpose()
    return mat

In [13]:
def generate(h, cur_in):
    x = cur_in
    mm1 = tf.matmul(Wxh, x)
    mm2 = tf.matmul(Whh, h)
    h_next = tf.tanh(mm1 + mm2 + bh)
    logit = tf.matmul(Why, h) + by
    pred = tf.nn.softmax(logit)
    pred_char = chars[tf.argmax(pred).eval(session=sess)[0]]
    return h_next, pred_char

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(num_epochs):
    random.shuffle(word_list)
    _current_state = np.zeros((hidden_size, batch_size))
    for word in word_list:
        num_batches = (len(word) - 1) - (seq_length)
        for j in range(num_batches):
            word_in = word[j:j+seq_length]
            mat_in = word_mat(word_in)
            mat_in = np.reshape(mat_in, (vocab_size, seq_length))
            word_out = word[j+1:j+seq_length+1]
            mat_out = word_mat(word_out)
            mat_out = np.reshape(mat_out, (vocab_size, seq_length))
            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
            [total_loss, train_step, current_state, predictions_series],
            feed_dict={
                X:mat_in,
                y_:mat_out,
                init_state:_current_state
            })
    print(_total_loss)
    generated = []
    state = np.zeros((hidden_size, 1),dtype=np.float32)
    cur_char = one_hot('x')
    generated.append(chars[list(cur_char).index(1)])

    for i in range(10):
        state, cur_char = generate(state, cur_char)
        generated.append(cur_char)
        cur_char = one_hot(cur_char)

    print(''.join([str(g) for g in generated]))
